In [1]:
%matplotlib qt
import pyxem as pxm
import hyperspy.api as hs
from pathlib import Path
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
#import seaborn as sns
from math import sqrt

## Assist functions

In [24]:
def wavelength(V, m0=9.1093837015*1e-31, e=1.60217662*1e-19, h=6.62607004*1e-34 , c=299792458):
    """
    Return the wavelength of an accelerated electron in [Å]
    
    Arguments
    ---------
    m0 : float, Rest mass of electron [kg]
    e : float, Elementary charge of electron [C]
    h : float, Planck' constant [m^2 kg/s]
    c : float, Speed of light in vacuum [m/s]
    """
    
    return h / sqrt( 2 * m0 * e * V * ( 1.0 + ( e*V / ( 2*m0*c**2 ) ) ) ) * 1E10

def camera_length(r, d, wavelength, pixel_size = 55):
    """
    Return the camera length in cm
    
    Arguments
    ---------
    r : float, Distance on detector [px]
    d : float, Interplanar distance in crystal [Å]
    wavelength : float, Wavelength of electrons [Å]
    pixel_size : float, Physical size of pixels on detector [um]. Default is 55 um, the pixel size on the Merlin Detector
    """
    r = float(r)
    d = float(d) * 1E-10 #convert to [m]
    wavelength = float(wavelength) * 1E-10 #convert to [m]
    pixel_size = float(pixel_size) * 1E-6 #convert to [m]
    
    return r * pixel_size * d / wavelength * 1E2

## Load data

In [83]:
signal = pxm.load(r'/media/emilc/Data/Merlin/NanoTools2020/Nanotools/Sample3/SAED_100cm.hspy')

ValueError: No file name matches this pattern

### Diffraction

In [186]:
cl = 25
#filename = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\2020_09_12_NanowireSC58A8_Merlin\NWX\2020_09_12_NanowireSC58A8_Merlin_calibrations\PED_8cm_1p08deg_NBDa5Spot1nm_2.mib')
filename = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\2020_09_12_NanowireSC58A8_MerlinCalibrations\NWX\2020_09_12_NanowireSC58A8_Merlin_calibrations\SAEDP_{cl:.0f}cm_NBDa5Spot1nm.mib'.format(cl=cl))

metadata = {
    'Acquisition_instrument':{
        'TEM':{
            'mode': 'NBD',
            'alpha': 'Alpha 5',
            'spot': 1,
            'cameralength': cl*1E-2,
            'acceleration_voltage': 200E3,
        }
    },
    'Session':{
        'Date': str(dt.datetime.today),
        'Operator': 'Emil Christiansen',
        'Specimen': 'GaAs/GaAsSb'
    }
}

signal = pxm.load_mib(str(filename))
data_array = signal.data
data_array=data_array.reshape((256, 256))
siganl = pxm.ElectronDiffraction2D(data_array, metadata=signal.metadata.as_dictionary())
signal.save(str(filename.with_suffix('.hspy')))

signal = pxm.ElectronDiffraction2D(pxm.load(str(filename.with_suffix('.hspy'))))
signal.metadata.add_dictionary(metadata)

This mib file appears to be TEM data. The stack is returned with no reshaping.
Overwrite 'C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\2020_09_12_NanowireSC58A8_MerlinCalibrations\NWX\2020_09_12_NanowireSC58A8_Merlin_calibrations\SAEDP_25cm_NBDa5Spot1nm.hspy' (y/n)?
y


### TEM

In [ ]:
filename = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\2020_09_12_NanowireSC58A8_Merlin\NWX\2020_09_12_NanowireSC58A8_Merlin_calibrations\BF_40kx_NBDa5Spot1nm.mib')
signal = pxm.load_mib(str(filename))
data_array = signal.data
data_array=data_array.reshape((256, 256))
signal = hs.signals.Signal2D(data_array, metadata=signal.metadata.as_dictionary())
signal.save(str(filename.with_suffix('.hspy')))
signal = hs.load(str(filename.with_suffix('.hspy')))

## Add metadata

In [4]:
metadata = {
    'Acquisition_instrument':{
        'TEM':{
            'mode': 'NBD',
            'alpha': 'Alpha 5',
            'spot': '1.0',
            'cameralength': '8',
            'acceleration_voltage': '200',
            'rocking_angle': 1.08,
            'rocking_frequency': 100
        }
    },
    'Session':{
        'Date': str(dt.datetime.today),
        'Operator': 'Emil Christiansen',
        'Specimen': 'GaAs/GaAsSb'
    }
}
signal.metadata.add_dictionary(metadata)

## Calibrate

### Precession (Ring fitting)

In [6]:
signal.plot(norm='log')

/home/emilc/anaconda3/envs/mib2hspyV2/lib/python3.8/site-packages/hyperspy/drawing/image.py:474: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  self.ax.imshow(data,


In [ ]:
g = hs.model.components1D.Expression(
    expression="height * exp(-(x - x0) ** 2 * 4 * log(2)/ fwhm ** 2)",
    name="Gaussian",
    position="x0",
    height=1,
    fwhm=1,
    x0=0,
    module="numpy")

### Line calibration

In [80]:
signal.plot(norm='log')
line = pxm.roi.Line2DROI(0, 0, 1, 1, linewidth=1)
line.add_widget(signal)

/home/emilc/anaconda3/envs/mib2hspyV2/lib/python3.8/site-packages/hyperspy/drawing/image.py:474: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  self.ax.imshow(data,
/home/emilc/anaconda3/envs/mib2hspyV2/lib/python3.8/site-packages/hyperspy/drawing/_widgets/line2d.py:252: MatplotlibDeprecationWarning: Setting the line's pick radius via set_picker is deprecated since 3.3 and will be removed two minor releases later; use set_pickradius instead.
  self.patch = self.ax.plot(
/home/emilc/anaconda3/envs/mib2hspyV2/lib/python3.8/site-packages/hyperspy/drawing/_widgets/line2d.py:268: MatplotlibDeprecationWarning: Setting the line's pick radius via set_picker is deprecated since 3.3 and will be removed two minor releases later; use set_pickradius instead.
  wi, = self.ax.plot(
/home/emilc/anaconda3/envs/mib2hspyV2/lib/python

In [81]:
profile = line(signal)
profile.plot(norm='log')
span = pxm.roi.SpanROI(1, 2)
#span.add_widget(profile, axes=[1,2])
span.add_widget(profile)

### Magnification Calibration

In [ ]:
units = 'nm'
l = 132.1 #Physical distance/feature
r = span.right-span.left #Distance on detector

lx = l * np.cos(line.angle() * np.pi / 180) #physical distance in x
rx = r * np.cos(line.angle() * np.pi / 180) #detector distance along x
ly = l * np.sin(line.angle() * np.pi / 180) #physical distance in y
ry = r * np.sin(line.angle() * np.pi / 180) #detector distance along y

scale_x = abs(lx/rx)
scale_y = abs(ly/ry)
print('Real distance: {l:.2e} nm\nDetector distance: {r:.2e} px\nScale x: {scale_x:.2e} nm/px\nScale y: {scale_y:.2e}'.format(l=l, r=r, scale_x=scale_x, scale_y=scale_y))

signal.axes_manager[0].name = 'x'
signal.axes_manager[1].name = 'y'
signal.axes_manager['x'].scale = scale_x
signal.axes_manager['y'].scale = scale_y
signal.axes_manager['x'].units = units
signal.axes_manager['y'].units = units

signal.plot()
line = pxm.roi.Line2DROI(50, 50, 100, 100, linewidth=10)
line.add_widget(signal)

In [ ]:
profile = line(signal)
profile.plot()
span = pxm.roi.SpanROI(20, 80)
span.add_widget(profile)

In [ ]:
signal.metadata.add_dictionary(metadata)
signal.save(str(filename.with_suffix('.hspy')))

### Diffraction Calibration

In [82]:
h, k, l = 1,1,1
n = 1
nh, nk, nl, = n, n, n
H, K, L = nh*h, nk*k, nl*l
x0 = span.left#9.5 #px
x1 = span.right#253.6
r = span.right - span.left
a = 5.431020511# Å Silicon
#a = 5.65325#Å
b = a
c = a
acceleration_voltage = 200E3 #V

g = np.array([H/a, K/b, L/c])
g = np.sqrt(np.sum(g**2))
d = 1 / g
scale = g/r
angular_scale_mrad = scale * wavelength(acceleration_voltage) *1000
angular_scale_deg = angular_scale_mrad / 1000 * 180 / np.pi

detector_size_Å = 256*scale
detector_size_mrad = 256*angular_scale_mrad
detector_size_deg = 256*angular_scale_deg

cl = camera_length(r, d, wavelength(acceleration_voltage))
print('Real distance between ({h} {k} {l}) planes: {d:.2e} Å\nScattering vector length: {g:.2e} 1/Å \nScattering vector on detector: {r:.1f} px\nScale: {scale:.4f} 1/Å / px\nAngular scale ({acc:.0f} kV): {ang_scale_mrad:.4f} mrad / px\nAngular scale ({acc:.0f} kV): {ang_scale_deg:.4f} deg / px\nDetector sizes: {det_size_Å:.3f} Å^-1, {det_size_mrad:.3f} mrad, {det_size_deg:.3f} deg\nCamera length ({acc:.0f} kV): {cl:.3f} cm'.format(h=H, k=K, l=L, d=d, g=g, r=r, scale=scale, ang_scale_mrad = angular_scale_mrad, ang_scale_deg=angular_scale_deg, det_size_Å=detector_size_Å, det_size_mrad=detector_size_mrad, det_size_deg=detector_size_deg, acc=acceleration_voltage*1E-3, cl=cl))

Real distance between (1 1 1) planes: 3.14e+00 Å
Scattering vector length: 3.19e-01 1/Å 
Scattering vector on detector: 198.9 px
Scale: 0.0016 1/Å / px
Angular scale (200 kV): 0.0402 mrad / px
Angular scale (200 kV): 0.0023 deg / px
Detector sizes: 0.411 Å^-1, 10.297 mrad, 0.590 deg
Camera length (200 kV): 136.743 cm


In [41]:
cl

27.475417115783806

In [15]:
r

186.95450326305004

In [16]:
d

0.39195014425002855

In [17]:
wavelength(acceleration_voltage)

0.025079340161710043

In [18]:
cl * 100

0.0016069906236493134

#### Add as metadata

In [173]:
signal.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── acceleration_voltage = 200000.0
│       ├── alpha = Alpha 5
│       ├── cameralength = 0.15
│       ├── mode = NBD
│       └── spot = 1
├── General
│   └── title = 
├── Session
│   ├── Date = <built-in method today of type object at 0x00007FF91B74B530>
│   ├── Operator = Emil Christiansen
│   └── Specimen = GaAs/GaAsSb
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

In [174]:
signal.metadata.add_dictionary({'Acquisition_instrument': {'TEM': {'calibrations': {'Scale': scale, 'Angular_scale': angular_scale, 'Actual_camera_length': cl}}}})

In [327]:
from math import nan

In [328]:
nan

nan

In [339]:
s = 'a'

In [343]:
s.replace()

TypeError: 'str' object does not support item assignment

In [342]:
s

'a'

In [354]:
from math import isnan
    

In [356]:
isnan(None)

TypeError: must be real number, not NoneType

In [345]:
f = foo()

In [353]:
class foo:
    
    def __init__(self):
        self.a = [1]
        self.b = [2]
    
    def __iter__(self):
        parameters = [self.a, self.b]
        for parameter in parameters:
            yield parameter
    def __str__(self):
        return 'a={self.a}, b={self.b}'.format(self=self)
    
f = foo()
print(f)
for v, p in enumerate(f):
    print(v)
    p.append(v)
print(f)

a=[1], b=[2]
0
1
a=[1, 0], b=[2, 1]


In [348]:
f.a

1

In [349]:
f.b

2

In [329]:
test = 'Test Navn'

In [332]:
test.replace(' ', '_').lower()

'test_navn'

In [331]:
test

'Test Navn'

In [175]:
signal.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── acceleration_voltage = 200000.0
│       ├── alpha = Alpha 5
│       ├── calibrations
│       │   ├── Actual_camera_length = 27.301134893108742
│       │   ├── Angular_scale = 0.2883095895391469
│       │   └── Scale = 0.00803278018356916
│       ├── cameralength = 0.15
│       ├── mode = NBD
│       └── spot = 1
├── General
│   └── title = 
├── Session
│   ├── Date = <built-in method today of type object at 0x00007FF91B74B530>
│   ├── Operator = Emil Christiansen
│   └── Specimen = GaAs/GaAsSb
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

#### Find centre of direct beam manually

In [176]:
signal.plot()
roi = pxm.roi.CircleROI(128, 128, 10)
roi.add_widget(signal)

C:\Users\emilc\Miniconda3\envs\pyXem\lib\site-packages\hyperspy\drawing\signal1d.py:123: UserWarning: Attempting to set identical left == right == 0.0 results in singular transformations; automatically expanding.
  plt.xlim(np.min(x_axis_lower_lims), np.max(x_axis_upper_lims))
C:\Users\emilc\Miniconda3\envs\pyXem\lib\site-packages\hyperspy\drawing\_widgets\vertical_line.py:34: MatplotlibDeprecationWarning: Setting the line's pick radius via set_picker is deprecated since 3.3 and will be removed two minor releases later; use set_pickradius instead.
  self.patch = [ax.axvline(self._pos[0],


## Precession  calibration

In [37]:
signal.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── acceleration_voltage = 200
│       ├── alpha = Alpha 5
│       ├── cameralength = 8
│       ├── mode = NBD
│       ├── rocking_angle = 1.08
│       ├── rocking_frequency = 100
│       └── spot = 1.0
├── General
│   └── title = 
├── Session
│   ├── Date = <built-in method today of type object at 0x00007FFFADF9B530>
│   ├── Operator = Emil Christiansen
│   └── Specimen = GaAs/GaAsSb
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

In [40]:
#Diffraction pattern (span) should be calibrated correctly already
d = span.right-span.left #Diameter of ring
r = d/2
wavelength = 0.0251 #Å
phi = r*wavelength #rad
Phi = phi*180/np.pi
print('Precession angle: {phi:.3e} mrad\nPrecession angle: {Phi:.3e} deg'.format(phi=phi*1000, Phi=Phi))
signal.metadata.Acquisition_instrument.TEM['rocking_angle_true_deg'] = Phi
signal.metadata.Acquisition_instrument.TEM['rocking_angle_true_rad'] = phi
signal.save(str(filename.with_suffix('.hspy')))

Precession angle: 3.416e+01 mrad
Precession angle: 1.957e+00 deg
Overwrite 'C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\2020_09_12_NanowireSC58A8_Merlin\NWX\2020_09_12_NanowireSC58A8_Merlin_calibrations\PED_8cm_1p08deg_NBDa5Spot1nm_2.hspy' (y/n)?
y


### Apply calibrations

In [177]:
centre = (roi.cx, roi.cy)
signal.set_diffraction_calibration(scale, center = centre*np.array(scale))

In [139]:
signal.plot(norm='log')

C:\Users\emilc\Miniconda3\envs\pyXem\lib\site-packages\hyperspy\drawing\signal1d.py:123: UserWarning: Attempting to set identical left == right == 0.0 results in singular transformations; automatically expanding.
  plt.xlim(np.min(x_axis_lower_lims), np.max(x_axis_upper_lims))
C:\Users\emilc\Miniconda3\envs\pyXem\lib\site-packages\hyperspy\drawing\_widgets\vertical_line.py:34: MatplotlibDeprecationWarning: Setting the line's pick radius via set_picker is deprecated since 3.3 and will be removed two minor releases later; use set_pickradius instead.
  self.patch = [ax.axvline(self._pos[0],
C:\Users\emilc\Miniconda3\envs\pyXem\lib\site-packages\hyperspy\drawing\image.py:474: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  self.ax.imshow(data,


## Save calibrated data

In [168]:
signal.save(str(filename.with_suffix('.hspy')))

Overwrite 'C:\Users\emilc\OneDrive - NTNU\NORTEM\Merlin\2020_09_12_NanowireSC58A8_MerlinCalibrations\NWX\2020_09_12_NanowireSC58A8_Merlin_calibrations\SAEDP_12cm_NBDa5Spot1nm.hspy' (y/n)?
y


# Inspect / get calibrations

In [242]:
signal = pxm.load(r'NWX/2020_09_12_NanowireSC58A8_Merlin_calibrations/SAEDP_8cm_NBDa5Spot1nm.hspy')

scale = signal.axes_manager['x'].scale
unit = signal.axes_manager['x'].units
size = 256*scale
pixel_size = 55E-6 #pixel size
physical_detector_size = 256*pixel_size*1E9 #nm 
magnification = physical_detector_size / size
print('Magnification: {mag:.0f}\nScale: {scale:.2f} {unit}/px\nDetector size: {size:.2f} {unit}'.format(mag=magnification, scale=scale, unit=unit, size=size))

ValueError: There is no DataAxis named x

In [288]:
signal.axes_manager

<Axes manager, axes: (1|256, 256)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
     <undefined> |      1 |      0 |       0 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ 
              kx |    256 |        |    -1.8 |   0.014 | $A^{-1}$ 
              ky |    256 |        |    -1.7 |   0.014 | $A^{-1}$

In [324]:
d = {'A': {'a':1}, 'B': {'b': 2}}

In [326]:
d.get('A').get('a')

1

In [322]:
None * 2

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

In [294]:
isinstance(signal, hs.signals.BaseSignal)

True

In [321]:
float(signal.max(np.arange(0, len(signal.data.shape), 1)).data)

41806.0

In [312]:
signal.max()

<ElectronDiffraction2D, title: , dimensions: (|256, 256)>

In [308]:
float(signal.max(axis=[0, 1, 2]).data)

TypeError: float() argument must be a string or a number, not 'BaseSignal'

In [297]:
signal.data.dtype

dtype('>u4')

In [290]:
signal.axes_manager[0].units = ''

In [292]:
signal.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── acceleration_voltage = 200000.0
│       ├── alpha = Alpha 5
│       ├── calibrations
│       │   ├── Actual_camera_length = 15.583701666071962
│       │   ├── Angular_scale = 0.00035293283443524337
│       │   └── Scale = 0.01407265231698897
│       ├── cameralength = 0.08
│       ├── mode = NBD
│       └── spot = 1
├── General
│   └── title = 
├── Session
│   ├── Date = <built-in method today of type object at 0x00007FF91B74B530>
│   ├── Operator = Emil Christiansen
│   └── Specimen = GaAs/GaAsSb
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

In [278]:
s.axes_manager[0]

<Unnamed 0th axis, size: 1, index: 0>

In [285]:
s_2 = hs.signals.Signal2D(data, lazy=True)

In [287]:
s_2.

array([[[3, 6, 5, ..., 8, 4, 6],
        [8, 9, 7, ..., 3, 5, 8],
        [8, 9, 1, ..., 2, 4, 4],
        ...,
        [4, 2, 7, ..., 3, 5, 4],
        [8, 3, 4, ..., 8, 9, 5],
        [4, 5, 6, ..., 6, 5, 3]]], dtype=uint32)

In [280]:
data = s.data

In [281]:
data.astype('bool')

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]]])

In [277]:
s.axes_manager.trait_set(axis_0={'name': 'test'})

<Axes manager, axes: (1|256, 256)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
     <undefined> |      1 |      0 |       0 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ 
              kx |    256 |        |    -1.8 |   0.014 | $A^{-1}$ 
              ky |    256 |        |    -1.7 |   0.014 | $A^{-1}$

In [270]:
s.axes_manager.trait_set({'axis-0': {'name': 'test'}})

<Axes manager, axes: (1|256, 256)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
     <undefined> |      1 |      0 |       0 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ 
              kx |    256 |        |    -1.8 |   0.014 | $A^{-1}$ 
              ky |    256 |        |    -1.7 |   0.014 | $A^{-1}$

In [267]:
s.axes_manager

<Axes manager, axes: (1|256, 256)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
     <undefined> |      1 |      0 |       0 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ 
              kx |    256 |        |    -1.8 |   0.014 | $A^{-1}$ 
              ky |    256 |        |    -1.7 |   0.014 | $A^{-1}$

In [257]:
s = signal.deepcopy()

In [262]:
s 

array([[[3, 6, 5, ..., 8, 4, 6],
        [8, 9, 7, ..., 3, 5, 8],
        [8, 9, 1, ..., 2, 4, 4],
        ...,
        [4, 2, 7, ..., 3, 5, 4],
        [8, 3, 4, ..., 8, 9, 5],
        [4, 5, 6, ..., 6, 5, 3]]], dtype=uint32)

In [258]:
data_array = s.data

In [259]:
data_array.reshape(256, 256)

array([[3, 6, 5, ..., 8, 4, 6],
       [8, 9, 7, ..., 3, 5, 8],
       [8, 9, 1, ..., 2, 4, 4],
       ...,
       [4, 2, 7, ..., 3, 5, 4],
       [8, 3, 4, ..., 8, 9, 5],
       [4, 5, 6, ..., 6, 5, 3]], dtype=uint32)

In [255]:
signal.axes_manager.as_dictionary()

{'axis-0': {'name': <undefined>,
  'scale': 1.0,
  'offset': 0.0,
  'size': 1,
  'units': <undefined>,
  'navigate': True},
 'axis-1': {'name': 'ky',
  'scale': 0.01407265231698897,
  'offset': -1.7309362349896433,
  'size': 256,
  'units': '$A^{-1}$',
  'navigate': False},
 'axis-2': {'name': 'kx',
  'scale': 0.01407265231698897,
  'offset': -1.801299496574588,
  'size': 256,
  'units': '$A^{-1}$',
  'navigate': False}}

In [253]:
len(signal.axes_manager.shape)

3

In [254]:
signal.axes_manager

<Axes manager, axes: (1|256, 256)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
     <undefined> |      1 |      0 |       0 |       1 | <undefined> 
---------------- | ------ | ------ | ------- | ------- | ------ 
              kx |    256 |        |    -1.8 |   0.014 | $A^{-1}$ 
              ky |    256 |        |    -1.7 |   0.014 | $A^{-1}$

In [245]:
a = {'test_1/Å': 2}

In [246]:
a

{'test_1/Å': 2}

In [247]:
2 != np.nan

True

In [248]:
2 is np.nan

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-248-7f9587bafb1a>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  2 is np.nan


False

In [244]:
signal.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── acceleration_voltage = 200000.0
│       ├── alpha = Alpha 5
│       ├── calibrations
│       │   ├── Actual_camera_length = 15.583701666071962
│       │   ├── Angular_scale = 0.00035293283443524337
│       │   └── Scale = 0.01407265231698897
│       ├── cameralength = 0.08
│       ├── mode = NBD
│       └── spot = 1
├── General
│   └── title = 
├── Session
│   ├── Date = <built-in method today of type object at 0x00007FF91B74B530>
│   ├── Operator = Emil Christiansen
│   └── Specimen = GaAs/GaAsSb
└── Signal
    ├── binned = False
    └── signal_type = electron_diffraction

In [217]:
signal.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── acceleration_voltage = 200
│       ├── alpha = Alpha 5
│       ├── magnification = 10000.0
│       ├── mode = NBD
│       └── spot = 1.0
├── General
│   └── title = 
├── Session
│   ├── Date = <built-in method today of type object at 0x00007FFFADF9B530>
│   ├── Operator = Emil Christiansen
│   └── Specimen = GaAs/GaAsSb
└── Signal
    ├── binned = False
    ├── exposure_time = None
    ├── flyback_times = None
    ├── frames_number_skipped = None
    ├── scan_X = None
    └── signal_type = TEM

In [218]:
scale = signal.axes_manager['x'].scale
unit = signal.axes_manager['x'].units
size = 256*scale
pixel_size = 55E-6 #pixel size
physical_detector_size = 256*pixel_size*1E9 #nm 
magnification = physical_detector_size / size
print('Magnification: {mag:.0f}\nScale: {scale:.2f} {unit}/px\nDetector size: {size:.2f} {unit}'.format(mag=magnification, scale=scale, unit=unit, size=size))

Magnification: 18483
Scale: 2.98 nm/px
Detector size: 761.78 nm


In [145]:
angular_scale*1000

0.35293283443524337

In [205]:
nominal_cl = [8, 10, 12, 15, 20, 25]
actual_cl = [16.584, 19.077, 22.314, 27.301, 35.468, 43.847]
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(nominal_cl, actual_cl, '--x')
ax.set_xlabel('Nominal camera length (cm)')
ax.set_ylabel('Actual camera length (cm)')
ax.set_title('Merlin camera lengths at 200 kV')
for ncl, acl in zip(nominal_cl, actual_cl):
    ax.annotate('{:.2f}'.format(acl), xy=(ncl, acl))
#ax.set_xlim(7, 26)

In [233]:
nominal_mag = [8E3, 10E3, 12E3, 15E3, 20E3, 25E3, 30E3, 40E3]
actual_mag = [14918, 18483, 21624, 26827, 35247, 43131, 53735, 69635]
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(nominal_mag, actual_mag, '--x')
ax.set_xlabel('Nominal magnification')
ax.set_ylabel('Actual magnification')
ax.set_title('Merlin magnifications in Mag1 at 200 kV')
for nmag, amag in zip(nominal_mag, actual_mag):
    ax.annotate('{:.0f}'.format(amag), xy=(nmag, amag))
#ax.set_xlim(7, 26)